In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap
# !{sys.executable} -m pip install pip rasterstats 

In [ ]:
import ee
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import io
# from rasterstats import zonal_stats
import fiona
import rasterio.mask
import geemap
import glob
import boto3

In [ ]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

# Read input data

In [ ]:
# define directory
out_dir = os.getcwd()
bucket_name = 'cities-urbanshift' 
aws_s3_dir = 'https://'+bucket_name+'.s3.eu-west-3.amazonaws.com'

In [ ]:
# read forest carbon flux data
# paper: https://www.nature.com/articles/s41558-020-00976-6 
# on GEE: https://code.earthengine.google.com/b21e2ea3103b09710e5ff5004e6d2617

netFlux = ee.ImageCollection('projects/wri-datalab/gfw-data-lake/net-flux-forest-extent-per-ha-v1-2-2-2001-2021/net-flux-global-forest-extent-per-ha-2001-2021')
net_flux = netFlux.reduce(ee.Reducer.mean()); # for use in visualizations
net_fluxw0 = net_flux.unmask(0) # unmask to 0 to enable pixels without tree carbon flux to be included in mean calculation
fluxScale = net_flux.projection().nominalScale()
# Map.addLayer(net_flux,{palette:['green','white','red']})
# Map.addLayer(net_fluxw0,{palette:['green','white','red']},"",0)


In [ ]:
# get list of c4f cities
boundary_georef = pd.read_csv(aws_s3_dir+'/data/boundaries/v_0/boundary_georef.csv')
boundary_georef

# Compute indicator

In [ ]:
this_indicator = pd.DataFrame() 

In [ ]:
# define calcuation function to get pixel counts, convert to percents and append to data frame
def CountCalcs(FC,DF):
    # reduce images to get mean flux by region
    sums = net_fluxw0.reduceRegions(FC,ee.Reducer.mean().setOutputs(['meanFluxMgcO2eperHAperiod']),30) # original dataset at 30m resolution
    
    # convert pixel counts to area percentages and saves to FC as property
    def toPct(feat):
        periodyears = (2021 - 2001) + 1 # number of years to average over average over 2001-2019 inclusive
        periodvalue = feat.getNumber('meanFluxMgcO2eperHAperiod')
        # methods to handle null values: https://gis.stackexchange.com/questions/341685/handling-null-inside-map-in-google-earth-engine
        annualFlux = ee.Algorithms.If(periodvalue,(periodvalue.divide(periodyears)),"NA") 
        return feat.set({
            'meanFluxMgcO2eperHAannual': annualFlux
      })

    sums = sums.map(toPct).select(['geo_id','meanFluxMgcO2eperHAannual'])

    # store in df and apend
    df = geemap.ee_to_pandas(sums)
    df = df.rename(columns={'meanFluxMgcO2eperHAannual': 'GRE_5_2_meanannualTreeCarbonFluxMgcO2eperHA'})
    DF = DF.append(df)
    return DF

In [ ]:
for i in range(0,len(boundary_georef)):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']
        
    # process aoi level ------
    print("\n boundary_id_aoi: "+boundary_id_aoi)
    # read boundaries
    boundary_path = aws_s3_dir +'/data/boundaries/v_0/boundary-'+boundary_id_aoi+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    this_indicator = CountCalcs(boundary_geo_ee,this_indicator)
    
    # process unit of analysis level ------
    print("\n boundary_id_unit: "+boundary_id_unit)
    # read boundaries
    boundary_path = aws_s3_dir +'/data/boundaries/v_0/boundary-'+boundary_id_unit+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    this_indicator = CountCalcs(boundary_geo_ee,this_indicator)


In [ ]:
this_indicator#.tail(50)

# Merge with indicator table

In [ ]:
# read indicator table
cities_indicators = pd.read_csv(aws_s3_dir + '/indicators/cities_indicators_ericV1.csv')
cities_indicators

In [ ]:
def merge_indicators(indicator_table, new_indicator_table, indicator_name):
    if indicator_name in indicator_table.columns:
        print("replace with new calculations")
        indicator_table.drop(indicator_name, inplace=True, axis=1)
        new_indicator_table = new_indicator_table.drop_duplicates()
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left',
                                                     validate='one_to_many')
    else:
        print("add new indicators")
        new_indicator_table = new_indicator_table.drop_duplicates()
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left',
                                                     validate='one_to_many')
    return(cities_indicators_df)

In [ ]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators,
                                            new_indicator_table = this_indicator,
                                            indicator_name = 'GRE_5_2_meanannualTreeCarbonFluxMgcO2eperHA')

In [ ]:
cities_indicators_merged

# Upload in aws s3

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
# upload to aws
key_data = 'indicators/cities_indicators_ericV1.csv'
cities_indicators_merged.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [ ]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')